<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    <b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b>
</h1>

---

In [ ]:
import numpy as np

---

## Верхнетреугольная матрица

Напишите код, чтобы получить **верхнетреугольную матрицу** (это такая матрица, у которой все числа под главной диагональю равны 0), обнулив все ненужные элементы.

Самое простое решение - загуглить, потому что почти для всего в `NumPy` есть готовое решение!

In [ ]:
def get_triangular_matrix(A):
    return np.triu(A)

In [ ]:
A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

get_triangular_matrix(A)

---

## Broadcasting

На самом деле, можно выполнять арифметические операции над массивами с несовпадающими размерами. Это называется [**broadcasting**](https://numpy.org/doc/stable/user/basics.broadcasting.html). В процессе броадкастингa `NumPy` пытается сделать `shape` обоих массивов одинаковым, чтобы выполнить нужную операцию.

Для этого проводится сравнение размерностей, начиная с последних размерностей. Размерности совместимы, если:

1. Количество элементов по этой размерности совпадает.
2. Количество элементов в одной из размернойстей равно 1.

Допустим, у нас было два массива, которые мы складываем:

```python
shape_1 = (1, 2, 1)
shape_2 = (2, 100)
```

Тогда можно считать, что после броадкастинга получатся массивы с `shape = (1, 2, 100)` и уже они сложатся.

In [ ]:
a = np.array([1, 2, 3])
b = np.array([
    [0, 1, 2],
    [3, 4, 5]
])

print(a.shape)
print(b.shape)
print((a + b).shape)

In [ ]:
# ещё один пример
a = a.reshape(1, 1, 3)
print(a.shape)
print(b.shape)
print((a + b).shape)

Вообще говоря, вместо использование **broadcating**'а лучше в явном виде прописывать все размерности. Если размерности обоих тензоров совпадают (например, они оба трёхмерные), тогда необходимо, чтобы соответствующие элементы кортежей формы либо совпадали, либо один из них был равен 1.

Пример:

In [ ]:
a + b[np.newaxis, :, :].shape

---

## Ромбик

Напишите код, чтобы в квадратной матрице $5 \times 5$ из нулей получить ромб из единиц, который касается середин сторон квадрата.

In [ ]:
A = np.zeros((5, 5))

row_dist = col_dist = np.abs(np.linspace(-2, 2, 5))

dist = row_dist[np.newaxis, :] + col_dist[:, np.newaxis]

B = (dist == 2).astype(int)
B

---

## Как правильно выбрать оси?

**Безобидная задача на кумулятивные суммы**

Дана матрица $M \times N$. Напишите функцию, которая возвращает вектор средних значений по вертикали.

In [ ]:
def vertical_means(A):
    return A.mean(axis=0)

In [ ]:
A = np.random.randint(0, 10, (5, 5))
print(A)

vertical_means(A)

**Проблемы:**

* Что такое вертикальная ось? 
* По какой оси необходимо суммировать?
* Как не ошибиться?

**Ответ:** Операции всегда производятся по той оси, которая **исчезнет** после применения операции.

---

## Батчи

Как вы, возможно, знаете, обучение нейронных сетей происходит последовательной подачей на вход нейронной сети объектов из обучающей выборки. Представим, например, что объекты - это картинки в формате RGB.

Чтобы нейронная сеть обучалась быстрее, объекты в неё подаются не по одному, а **батчами** из $N$ объектов. Итак, на вход нейронной сети подаётся четырёхмерный (!!!) тензор формы `(batch_size, num_channels, height, width)`. Наверное, вы уже убедились, что знать, где правильно ставить оси, просто необходимо.

Благо, есть простое решение: можно просто **НЕ ТРОГАТЬ** нулевую ось.

---

## Стандартизация картинки

На диске лежит файл `image_batch.npy` (здесь в ноутбуке мы просто генеируем батч с помощью функции `np.random.randint`). В нём лежит батч (массив) картинок в формате `NumPy`. Каждая картинка задана как трёхмерная матрица формата `(num_channels, width, height)`. Нужно стандартизировать каналы одного пикселя, т.е. сделать так, чтобы для каждого пикселя среднее по всем каналам стало равно $0$, а стандартное отклонение - $1$.

In [ ]:
def normalize_pictures(A):
    """
    param A: np.array[batch_size, num_channels, width, height]
    """
    
    mu = A.mean(axis=(2, 3))
    sigma = A.std(axis=(2, 3))
    
    return (A - mu[:, :, np.newaxis, np.newaxis]) / sigma[:, :, np.newaxis, np.newaxis]

In [ ]:
batch = np.random.randint(0, 256, (100, 3, 300, 300))

norm_batch = normalize_pictures(batch)

norm_batch.mean(axis=(2, 3)), norm_batch.std(axis=(2, 3))

---

## Стандартизация и транспонирование 

Задание то же самое, но нужно стандартизировать все пиксели внутри каждой картинки и сменить формат `(batch_size, num_channels, x_coord, y_coord)` на `(batch_size, x_coord, y_coord, num_channels)`. Такой формат обычно удобнее для разных вычислений и вообще выглядит более естественным, но GPU более эффективно работают с первым.

In [ ]:
def normalize_and_transpose_pictures(A):
    """
    param A: np.array[batch_size, num_channels, width, height]
    """
    
    np.moveaxis(A, 1, -1)
    
    mu = A.mean(axis=(1, 2))
    sigma = A.std(axis=(1, 2))
    
    return (A - mu[:, np.newaxis, np.newaxis, :]) / sigma[:, np.newaxis, np.newaxis, :]

In [ ]:
res_transposed = normalize_and_transpose_pictures(batch)
res_transposed.mean(axis=(1, 2)), res_transposed.std(axis=(1, 2))

---

## Сжатие фотографии

Теперь мы хотим снизить качество батча фотографий - разбить каждое изображение на квадратики $2 \times 2$ и усреднить внутри них значения по каждому каналу отдельно.

In [ ]:
import skimage.measure

img = plt.imread('img/rose.jpg') / 255

compressed = skimage.measure.block_reduce(img, (4, 4, 1), np.mean)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

ax[0].imshow(img)
ax[1].imshow(compressed)
plt.show()

Проделаем тоже самое, используя `NumPy`:

In [ ]:
def low_quality(batch_images, size):
    """
    param A: np.array[batch_size, width, height, num_channels]
    """
    
    pixelized = np.zeros((batch_images.shape[0], size, size, batch_images.shape[1] // size, batch_images.shape[2] // size, batch_images.shape[3]))
    
    for k in range(batch_images.shape[0]):
        for i in range(size):
            for j in range(size):
                pixelized[k, i, j, :, :, :] = batch_images[:, i::size, j::size, :]

    return pixelized.mean(axis=(1, 2))

Протестируем на реальной картинке:

In [ ]:
img = plt.imread('img/rose.jpg') / 255

compressed = low_quality(img[np.newaxis, :, :, :], 4)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

ax[0].imshow(img)
ax[1].imshow(compressed[0])
plt.show()

---

## Ридж

Дана квадратная матрица $A$ и массив $b$ соответствующей длины. Прибавьте элементы массива $b$ к главной диагонали матрицы $A$.

In [ ]:
def upgraded_plus(A, b):
    return A + np.diag(b)

In [ ]:
A = np.arange(16).reshape(4, 4)
b = np.ones(4).astype(int)

A, upgraded_plus(A, b)